In [ ]:
pip install moviepy==1.0.3

In [17]:
"""
SQL Video Generator
====================

Creates a short promo/educational video similar in style to an Instagram post.

Features
- Accepts user input (title, subtitle, list of bullet points) via interactive prompts or command-line args.
- Renders slides (PIL) with wrapped text, logo and social icons (optional).
- Produces narration audio using pyttsx3 (offline TTS) and mixes it with background music (optional).
- Assembles the final video using MoviePy with crossfade transitions.

Requirements
- Python 3.8+
- moviepy (`pip install moviepy`)
- pillow (`pip install pillow`)
- pyttsx3 (`pip install pyttsx3`)  # offline TTS
- (Optional) pydub for extra audio handling (`pip install pydub`) and ffmpeg installed on your system.

Notes
- Provide paths for: a TrueType font file, a logo image (e.g. switchtech.de logo), Instagram and Telegram icon images (small PNGs with transparent background).
- If pyttsx3 isn't suitable on your machine, you can replace the `generate_voice()` function with another TTS (gTTS requires internet).
- Output is `output_video.mp4` by default.

Usage
- Run interactively: `python sql_video_generator.py`
- Or pass `--title`, `--bullets` etc. (basic argparse support present)

"""

import os
import textwrap
import argparse
from typing import List
from PIL import Image, ImageDraw, ImageFont
from moviepy.editor import ImageClip, AudioFileClip, concatenate_videoclips, CompositeVideoClip
# from moviepy import VideoFileClip
import pyttsx3

# ---------- Configurable defaults ----------
VIDEO_SIZE = (1080, 1080)  # square for Instagram feed
FPS = 30
SLIDE_DURATION = 3.0  # seconds per slide (excluding transition)
TRANSITION_DURATION = 0.6
FONT_SIZE_TITLE = 72
FONT_SIZE_BULLET = 42
BG_COLOR = (12, 34, 64)  # dark blue-ish
TEXT_COLOR = (245, 245, 245)
ACCENT_COLOR = (0, 170, 255)
OUTPUT_FILENAME = "output_video.mp4"
VOICE_RATE = 160

# ---------- Helpers ----------

def wrap_text(text: str, font: ImageFont.FreeTypeFont, max_width: int) -> List[str]:
    words = text.split()
    lines = []
    cur = []
    for w in words:
        cur.append(w)
        if font.getsize(" ".join(cur))[0] > max_width:
            # remove last word, push line
            cur.pop()
            lines.append(" ".join(cur))
            cur = [w]
    if cur:
        lines.append(" ".join(cur))
    return lines


def create_slide_image(title: str, subtitle: str, bullets: List[str], logo_path: str, instagram_path: str, telegram_path: str, font_path: str) -> Image.Image:
    """Create one PIL Image representing a slide."""
    w, h = VIDEO_SIZE
    im = Image.new("RGBA", VIDEO_SIZE, BG_COLOR)
    draw = ImageDraw.Draw(im)

    # Load fonts
    title_font = ImageFont.truetype(font_path, FONT_SIZE_TITLE)
    bullet_font = ImageFont.truetype(font_path, FONT_SIZE_BULLET)

    # Title centered near top
    title_lines = wrap_text(title, title_font, int(w * 0.85))
    y = int(h * 0.12)
    for line in title_lines:
        tw, th = draw.textsize(line, font=title_font)
        draw.text(((w - tw) / 2, y), line, font=title_font, fill=TEXT_COLOR)
        y += th + 6

    # Subtitle below title
    if subtitle:
        sub_font = ImageFont.truetype(font_path, int(FONT_SIZE_TITLE * 0.45))
        sub_lines = wrap_text(subtitle, sub_font, int(w * 0.8))
        for line in sub_lines:
            tw, th = draw.textsize(line, font=sub_font)
            draw.text(((w - tw) / 2, y + 6), line, font=sub_font, fill=TEXT_COLOR)
            y += th + 6

    # Bullets on middle-left
    bullet_x = int(w * 0.12)
    bullet_y = int(h * 0.45)
    max_bullet_width = int(w * 0.7)
    for b in bullets:
        lines = wrap_text(b, bullet_font, max_bullet_width)
        # bullet dot
        draw.ellipse((bullet_x - 12, bullet_y + 8, bullet_x - 4, bullet_y + 16), fill=ACCENT_COLOR)
        for li in lines:
            draw.text((bullet_x + 8, bullet_y), li, font=bullet_font, fill=TEXT_COLOR)
            bullet_y += bullet_font.getsize(li)[1] + 6
        bullet_y += 6

    # Bottom text: call to action + website
    cta_font = ImageFont.truetype(font_path, 28)
    cta_text = "For more details, stay tuned with switchtech.de"
    tw, th = draw.textsize(cta_text, font=cta_font)
    draw.text(((w - tw) / 2, h - 90), cta_text, font=cta_font, fill=TEXT_COLOR)

    # Add logo top-left if provided
    if logo_path and os.path.exists(logo_path):
        logo = Image.open(logo_path).convert("RGBA")
        logo.thumbnail((140, 140), Image.LANCZOS)
        im.paste(logo, (int(w * 0.06), int(h * 0.02)), logo)

    # Add social icons bottom-left/right
    icon_size = 64
    if instagram_path and os.path.exists(instagram_path):
        ins = Image.open(instagram_path).convert("RGBA")
        ins.thumbnail((icon_size, icon_size), Image.LANCZOS)
        im.paste(ins, (int(w * 0.08), h - 80), ins)
    if telegram_path and os.path.exists(telegram_path):
        tel = Image.open(telegram_path).convert("RGBA")
        tel.thumbnail((icon_size, icon_size), Image.LANCZOS)
        im.paste(tel, (int(w * 0.08) + icon_size + 16, h - 80), tel)

    return im


def generate_voice(text: str, out_path: str):
    """Generate speech from text using pyttsx3 (offline)."""
    engine = pyttsx3.init()
    engine.setProperty('rate', VOICE_RATE)
    # Pick voice if available (optional)
    voices = engine.getProperty('voices')
    if voices:
        # prefer a neutral-ish voice
        engine.setProperty('voice', voices[0].id)
    engine.save_to_file(text, out_path)
    engine.runAndWait()


# ---------- Main assembly ----------

def build_video(title: str, subtitle: str, bullets: List[str], logo_path: str, instagram_path: str, telegram_path: str, font_path: str, music_path: str, output: str):
    slides = []
    # Create a cover slide (title only)
    cover_img = create_slide_image(title, subtitle, [], logo_path, instagram_path, telegram_path, font_path)
    cover_img_path = "slide_cover.png"
    cover_img.save(cover_img_path)
    slides.append(cover_img_path)

    # Create one slide per bullet
    for idx, b in enumerate(bullets, start=1):
        img = create_slide_image(title, subtitle, [b], logo_path, instagram_path, telegram_path, font_path)
        path = f"slide_{idx}.png"
        img.save(path)
        slides.append(path)

    # Create narration text
    narration_text = title + ". "
    if subtitle:
        narration_text += subtitle + ". "
    for i, b in enumerate(bullets, start=1):
        narration_text += f"Point {i}: {b}. "

    audio_path = "narration.mp3"
    print("Generating narration audio (offline TTS)...")
    generate_voice(narration_text, audio_path)

    # Build video clips
    clips = []
    for s in slides:
        img_clip = ImageClip(s).set_duration(SLIDE_DURATION).resize(VIDEO_SIZE)
        clips.append(img_clip)

    # Add simple crossfade transition by concatenating with padding
    final_clip = concatenate_videoclips(clips, method="compose", padding=-TRANSITION_DURATION)

    # Add background music if provided
    if music_path and os.path.exists(music_path):
        bg = AudioFileClip(music_path).volumex(0.15)
        # loop or cut to length
        bg = bg.audio_loop(duration=final_clip.duration)
    else:
        bg = None

    # add narration
    if os.path.exists(audio_path):
        voice = AudioFileClip(audio_path)
    else:
        voice = None

    # Mix audio
    if bg and voice:
        # use voice as main and lower bg
        final_audio = CompositeAudioClip([bg.set_start(0), voice.set_start(0)])
        final_clip = final_clip.set_audio(final_audio)
    elif voice:
        final_clip = final_clip.set_audio(voice)
    elif bg:
        final_clip = final_clip.set_audio(bg)

    print("Writing final video to", output)
    final_clip.write_videofile(output, fps=FPS, codec="libx264", audio_codec="aac")


# ---------- CLI / Interactive ----------

def parse_args():
    p = argparse.ArgumentParser(description="Generate a short SQL promo video from text input.")
    p.add_argument("--title", default=None)
    p.add_argument("--subtitle", default=None)
    p.add_argument("--bullets", default=None, help="semicolon-separated bullets, e.g. \"Basics;Advanced\"")
    p.add_argument("--logo", default="logo.png")
    p.add_argument("--instagram", default="instagram.png")
    p.add_argument("--telegram", default="telegram.png")
    p.add_argument("--font", default="arial.ttf", help="Path to a .ttf font file")
    p.add_argument("--music", default=None, help="Optional background music path")
    p.add_argument("--out", default=OUTPUT_FILENAME)
    return p.parse_args()


if __name__ == "__main__":
    args = parse_args()

    if not args.title:
        args.title = input("Enter the main title (e.g. 'We are starting SQL Course'): ")
    if not args.subtitle:
        args.subtitle = input("Enter a short subtitle (or leave blank): ")
    if not args.bullets:
        raw = input("Enter bullets separated by semicolons (;). Example: Theoretical concepts;Practical updates;Interview questions: ")
        args.bullets = raw

    bullets = [b.strip() for b in (args.bullets or "").split(";") if b.strip()]

    # sanity checks
    if not os.path.exists(args.font):
        print("Font path not found:", args.font)
        print("Please provide a valid .ttf font path via --font or place a font file named 'arial.ttf' in this folder.")
        raise SystemExit(1)

    # Build
    build_video(args.title, args.subtitle, bullets, args.logo, args.instagram, args.telegram, args.font, args.music, args.out)

    print("Done! Video saved to", args.out)


usage: ipykernel_launcher.py [-h] [--title TITLE] [--subtitle SUBTITLE] [--bullets BULLETS] [--logo LOGO]
                             [--instagram INSTAGRAM] [--telegram TELEGRAM] [--font FONT] [--music MUSIC] [--out OUT]
ipykernel_launcher.py: error: unrecognized arguments: -f C:\Users\LENOVO\AppData\Roaming\jupyter\runtime\kernel-3baa9338-8caf-42cc-96a0-363dfcef9504.json


SystemExit: 2

In [9]:
!pip install --upgrade pip

   ---------------------------------------- 0.0/1.8 MB ? eta -:--:--
   ---------------------------------------- 1.8/1.8 MB 24.0 MB/s eta 0:00:00


ERROR: To modify pip, please run the following command:
C:\Users\LENOVO\anaconda3\python.exe -m pip install --upgrade pip


In [11]:
!pip install moviepy pillow pyttsx3 pydub pywin32

In [15]:
!C:\Users\LENOVO\anaconda3\python.exe -m pip install --upgrade pip

  Using cached pip-25.2-py3-none-any.whl.metadata (4.7 kB)
Using cached pip-25.2-py3-none-any.whl (1.8 MB)
  Attempting uninstall: pip
    Found existing installation: pip 24.2
    Uninstalling pip-24.2:
      Successfully uninstalled pip-24.2


In [23]:
# SQL Promo/Educational Video Generator (Pillow v10+ safe, no MoviePy resize)
# Works in Jupyter Notebook

import os
from typing import List, Tuple
from PIL import Image, ImageDraw, ImageFont
from moviepy.editor import (
    ImageClip,
    AudioFileClip,
    concatenate_videoclips,
    CompositeAudioClip,
)

# ---------------- Config ----------------
VIDEO_SIZE: Tuple[int, int] = (1080, 1080)
FPS = 30
SLIDE_DURATION = 3.0
TRANSITION_DURATION = 0.6
FONT_SIZE_TITLE = 72
FONT_SIZE_SUB = 32
FONT_SIZE_BULLET = 42
FONT_SIZE_CTA = 28

BG_COLOR = (12, 34, 64)
TEXT_COLOR = (245, 245, 245)
ACCENT_COLOR = (0, 170, 255)

OUTPUT_FILENAME = "output_video.mp4"
VOICE_RATE = 160


# ---------- Safe thumbnail helper ----------
def safe_thumbnail(img, size):
    """Ensure compatibility with Pillow 10+ (ANTIALIAS removed)."""
    resample = getattr(Image, "Resampling", Image).LANCZOS
    img.thumbnail(size, resample)
    return img


# ---------------- Helpers ----------------
def _text_wh(draw: ImageDraw.ImageDraw, text: str, font: ImageFont.FreeTypeFont) -> Tuple[int, int]:
    if not text:
        return 0, 0
    l, t, r, b = draw.textbbox((0, 0), text, font=font)
    return r - l, b - t


def wrap_text_by_width(draw: ImageDraw.ImageDraw, text: str, font: ImageFont.FreeTypeFont, max_width: int) -> List[str]:
    words = text.split()
    lines, cur = [], []
    for w in words:
        cur.append(w)
        line = " ".join(cur)
        w_px, _ = _text_wh(draw, line, font)
        if w_px > max_width:
            cur.pop()
            if cur:
                lines.append(" ".join(cur))
            cur = [w]
    if cur:
        lines.append(" ".join(cur))
    return lines


def create_slide_image(
    title: str,
    subtitle: str,
    bullets: List[str],
    logo_path: str,
    instagram_path: str,
    telegram_path: str,
    font_path: str,
) -> Image.Image:
    # Create exactly VIDEO_SIZE so we never call MoviePy.resize()
    w, h = VIDEO_SIZE
    im = Image.new("RGBA", VIDEO_SIZE, BG_COLOR)
    draw = ImageDraw.Draw(im)

    title_font = ImageFont.truetype(font_path, FONT_SIZE_TITLE)
    sub_font = ImageFont.truetype(font_path, FONT_SIZE_SUB)
    bullet_font = ImageFont.truetype(font_path, FONT_SIZE_BULLET)
    cta_font = ImageFont.truetype(font_path, FONT_SIZE_CTA)

    # Title
    title_lines = wrap_text_by_width(draw, title, title_font, int(w * 0.85))
    y = int(h * 0.12)
    for line in title_lines:
        tw, th = _text_wh(draw, line, title_font)
        draw.text(((w - tw) // 2, y), line, font=title_font, fill=TEXT_COLOR)
        y += th + 6

    # Subtitle
    if subtitle:
        sub_lines = wrap_text_by_width(draw, subtitle, sub_font, int(w * 0.85))
        for line in sub_lines:
            tw, th = _text_wh(draw, line, sub_font)
            draw.text(((w - tw) // 2, y), line, font=sub_font, fill=TEXT_COLOR)
            y += th + 6

    # Bullets
    bullet_x = int(w * 0.12)
    bullet_y = int(h * 0.45)
    max_bullet_width = int(w * 0.72)
    for b in bullets:
        if not b:
            continue
        lines = wrap_text_by_width(draw, b, bullet_font, max_bullet_width)
        draw.ellipse((bullet_x - 14, bullet_y + 10, bullet_x - 4, bullet_y + 20), fill=ACCENT_COLOR)
        for li in lines:
            draw.text((bullet_x + 8, bullet_y), li, font=bullet_font, fill=TEXT_COLOR)
            _, lh = _text_wh(draw, li, bullet_font)
            bullet_y += lh + 6
        bullet_y += 6

    # CTA
    cta_text = "For more details, stay tuned with switchtech.de"
    tw, th = _text_wh(draw, cta_text, cta_font)
    draw.text(((w - tw) // 2, h - 90), cta_text, font=cta_font, fill=TEXT_COLOR)

    # Logo
    if logo_path and os.path.exists(logo_path):
        try:
            logo = Image.open(logo_path).convert("RGBA")
            safe_thumbnail(logo, (140, 140))
            im.paste(logo, (int(w * 0.06), int(h * 0.02)), logo)
        except Exception:
            pass

    # Social icons
    icon_size = 64
    x0 = int(w * 0.08)
    y0 = h - 82
    if instagram_path and os.path.exists(instagram_path):
        try:
            ins = Image.open(instagram_path).convert("RGBA")
            safe_thumbnail(ins, (icon_size, icon_size))
            im.paste(ins, (x0, y0), ins)
            x0 += icon_size + 16
        except Exception:
            pass
    if telegram_path and os.path.exists(telegram_path):
        try:
            tel = Image.open(telegram_path).convert("RGBA")
            safe_thumbnail(tel, (icon_size, icon_size))
            im.paste(tel, (x0, y0), tel)
        except Exception:
            pass

    return im


def generate_voice_wav(text: str, out_path: str, rate: int = VOICE_RATE) -> bool:
    """Return True if narration saved; False on any failure."""
    try:
        import pyttsx3
        engine = pyttsx3.init()
        engine.setProperty("rate", rate)
        voices = engine.getProperty("voices")
        if voices:
            engine.setProperty("voice", voices[0].id)
        engine.save_to_file(text, out_path)
        engine.runAndWait()
        return True
    except Exception:
        return False


def build_video(
    title: str,
    subtitle: str,
    bullets: List[str],
    logo_path: str,
    instagram_path: str,
    telegram_path: str,
    font_path: str,
    music_path: str = None,
    output: str = OUTPUT_FILENAME,
    no_voice: bool = False,
):
    slide_paths = []

    cover = create_slide_image(title, subtitle, [], logo_path, instagram_path, telegram_path, font_path)
    cover_path = "slide_cover.png"
    cover.save(cover_path)
    slide_paths.append(cover_path)

    for i, b in enumerate(bullets, 1):
        img = create_slide_image(title, subtitle, [b], logo_path, instagram_path, telegram_path, font_path)
        p = f"slide_{i}.png"
        img.save(p)
        slide_paths.append(p)

    narration = title.strip() + ". "
    if subtitle.strip():
        narration += subtitle.strip() + ". "
    for i, b in enumerate(bullets, 1):
        narration += f"Point {i}: {b.strip()}. "

    voice_ok = False
    voice_path = "narration.wav"
    if not no_voice and narration.strip():
        voice_ok = generate_voice_wav(narration, voice_path)

    # Build clips WITHOUT MoviePy.resize() to avoid ANTIALIAS usage internally
    clips = [ImageClip(p).set_duration(SLIDE_DURATION) for p in slide_paths]
    final_clip = concatenate_videoclips(clips, method="compose", padding=-TRANSITION_DURATION)

    # Audio
    bg = None
    if music_path and os.path.exists(music_path):
        try:
            bg = AudioFileClip(music_path).volumex(0.15).audio_loop(duration=final_clip.duration)
        except Exception:
            bg = None

    voice = None
    if voice_ok and os.path.exists(voice_path):
        try:
            voice = AudioFileClip(voice_path)
        except Exception:
            voice = None

    if bg and voice:
        final_clip = final_clip.set_audio(CompositeAudioClip([bg.set_start(0), voice.set_start(0)]))
    elif voice:
        final_clip = final_clip.set_audio(voice)
    elif bg:
        final_clip = final_clip.set_audio(bg)

    final_clip.write_videofile(output, fps=FPS, codec="libx264", audio_codec="aac")


# -------- Example run --------
title = "We are starting SQL Course"
subtitle = "Basics to Advanced • Hands-on • Interview Prep"
bullets = [
    "DDL vs DML: build and manipulate tables",
    "Joins overview: INNER, LEFT, RIGHT, FULL",
    "Indexes 101: speed vs write cost",
    "Window functions for analytics",
    "Real interview questions & patterns",
]

font_path = r"C:\Windows\Fonts\segoeui.ttf"  # adjust if needed
logo_path = "logo.png"
instagram_icon = "instagram.png"
telegram_icon = "telegram.png"
music_path = None  # optional background music file
output = "output_video.mp4"

build_video(
    title=title,
    subtitle=subtitle,
    bullets=bullets,
    logo_path=logo_path if os.path.exists(logo_path) else "",
    instagram_path=instagram_icon if os.path.exists(instagram_icon) else "",
    telegram_path=telegram_icon if os.path.exists(telegram_icon) else "",
    font_path=font_path,
    music_path=music_path,
    output=output,
    no_voice=False,
)

print("Done:", output)


Moviepy - Building video output_video.mp4.
MoviePy - Writing audio in output_videoTEMP_MPY_wvf_snd.mp4


MoviePy - Done.
Moviepy - Writing video output_video.mp4



Moviepy - Done !
Moviepy - video ready output_video.mp4
Done: output_video.mp4
